### Link to Google Drive: https://drive.google.com/drive/folders/1Pnuo1tB1XtiDjMa7eXmuUctL2XX9emU7
### Link to Tableau Online: https://dub01.online.tableau.com/#/site/telenordashboard/projects/123532
### Link to Google Cloud Storage: https://console.cloud.google.com/storage/browser?project=telenor-data-science
### Link to Google Cloud BigQuery: https://console.cloud.google.com/bigquery?project=telenor-data-science&p=telenor-data-science&d=telenor_dataset&page=dataset

In [ ]:
import glob
import os
import pandas as pd

In [ ]:
# Project info
PROJECT_ID = 'telenor-data-science'
DATASET = 'raw_dataset_joined.tar.gz'

In [ ]:
# Download dataset from GCS
!gsutil -q cp gs://{PROJECT_ID}/datasets/{DATASET} /tmp
!mkdir -p data
!tar -zxf /tmp/{DATASET} -C /content/data

In [ ]:
paths = glob.glob("/content/data/**/*.parquet", recursive=True)
dataframes = {}

# Load all parquet files as dataframes
for path in paths:
    df = pd.read_parquet(path)
    dir_name = path.split(os.sep)[-2]
    df_tuple = (df, os.path.splitext(path)[0][-4:])
    df_old = dataframes.get(dir_name, [])
    df_old.append(df_tuple)
    dataframes[dir_name] = df_old

# Create mapping {location -> [[df1, df2, df3, ...], [year1, year2, year3, ...]]}
for k, v in dataframes.items():
    dataframes[k] = list(zip(*v))

In [ ]:
# Print years and number of features
template = "{:20}\t{:>10}\t{}"
print(template.format("Location", "# columns", "Years"))
for location, data in dataframes.items():
    dfs, years = data
    years = ', '.join(sorted(years))

     # Count number of unique columns
    location_columns = set()
    for df in dfs:
        location_columns.update(df.columns)
    num_of_columns = len(location_columns)
    print(template.format(location, num_of_columns, years))


Location            	 # columns	Years
E6-Tiller           	        45	2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020
Alvim               	        41	2015, 2016, 2017, 2018, 2019, 2020
Sofienbergparken    	        58	2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020
Manglerud           	       104	2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020
Leiret              	        54	2016, 2017, 2018, 2019, 2020
Minnesundvegen, Gjøvik	        42	2014, 2015, 2016, 2017, 2018, 2019, 2020
Vangsveien, Hamar   	        43	2014, 2015, 2016, 2017, 2018, 2019, 2020
Rv 4, Aker sykehus  	       106	2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020
Ringsakervegen      	        30	2018, 2019, 2020
Bakke kirke         	        50	2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020
Mo

In [ ]:
'''
def rename_col(name):
    """Conversion to make column names compatible with BigQuery."""
    problematic_chars = ' .,:;{}()='
    for c in problematic_chars:
        name = name.replace(c, '_')
    return name
'''

import re
def rename_col(name):
    """Conversion to make column names compatible with BigQuery."""
    name = re.sub("(PRA\.(.+)\.(.+)\.(.+))", r'PRA_\3__\4', name)
    name = re.sub("(MET\.[a-zA-Z0-9]*:0\.)", r'MET_', name)
    name = re.sub(r"(NEA\..*\.(NOx|PM2_5|PM10|NO2|PM1|NO))", r'NEA_\2', name)
    name = re.sub(r"(Kystverket\.[a-zA-Z0-9_-]*\.(stationary|moving))", r'KV_\2', name)
    return name

def rename_file(name):
    name = name.lower()
    for c in " ,.-:;{}[]()=":
        name = name.replace(c, '_')
    return name

In [ ]:
base_dfs = []

for location, lists in dataframes.items():
    dfs, years = lists
    dfs = list(dfs)
    # Rename all columns (Thanks telenor :))
    for i, df in enumerate(dfs):
        translation = {col: rename_col(col) for col in df.columns}
        dfs[i] = df.rename(columns=translation)

    # Create new root dataframe for directory
    base_df = pd.DataFrame()

    # Add location and years data to dataframe
    base_df['year'] = None
    
    # Identify all unique columns
    location_columns = set()
    for df in dfs:
        location_columns.update(df.columns)
    
    # Generate colummns in base dataframe
    for column_name in location_columns:
        base_df[column_name] = None

    # Add all rows of data to base dataframe
    for df, year in zip(dfs, years):
        df['year'] = year
        base_df = base_df.append(df)
    
    # Add location to all rows
    base_df.insert(1, 'location', location)
    base_df = base_df.reindex(sorted(base_df.columns), axis=1)
    base_dfs.append(base_df)

In [ ]:
# Total number of rows in all datasets
import numpy as np
print(np.array([df.shape for df in base_dfs]).sum(axis=0))

[3700794    2557]


In [ ]:
# Export .parquet files to local storage
for location, base_df in zip(dataframes.keys(), base_dfs):
    print(base_df.columns)
    filename = rename_file(location)
    print(f"Exporting {filename} ...") 
    for col in columns:
        print(f"    {col}")
    !mkdir -p /tmp/location_dataset
    base_df.to_parquet(f"/tmp/location_dataset/{filename}.parquet")

In [ ]:
# Upload dataset to Google Cloud Storage
from google.colab import auth
auth.authenticate_user()

FROM = "/tmp/location_dataset"
TO = "gs://telenor-data-science/datasets/location_dataset"
IS_DIR = True

if IS_DIR:
    !gsutil -m cp -r {FROM} {TO}
else:
    !gsutil cp {FROM} {TO}

# Compress and upload compresed version to GCS
!tar -czvf /tmp/location_dataset.tar.gz /tmp/location_dataset
!gsutil cp /tmp/location_dataset.tar.gz gs://telenor-data-science/datasets/location_dataset.tar.gz

In [ ]:
# DOESNT WORK - Type conflict on parquet inferred schema vs. df index :( 
#root_df = pd.DataFrame()
root_df = base_dfs[0].copy().iloc[0:0]
#root_df = root_df.set_index(base_dfs[0].iloc[0:0].index)
# Identify all unique columns
root_columns = set()
for df in base_dfs:
    root_columns.update(df.columns)

# Generate colummns in root dataframe
for column_name in root_columns:
    root_df[column_name] = ''

# Rearrange columns
root_df = root_df.reindex(sorted(root_df.columns), axis=1)

#Change col types
#root_df.append(base_dfs[0])
#root_df = root_df.iloc[0:0]
root_df = root_df.astype(float)
root_df = root_df.astype({'year': 'str', 'location': 'str'})

root_df.to_parquet(f"/tmp/location_dataset/all_locations.parquet")
!gsutil cp /tmp/location_dataset/all_locations.parquet gs://telenor-data-science/datasets/location_dataset/all_locations.parquet
# Create table
!bq load --source_format=PARQUET --ignore_unknown_values --autodetect telenor-data-science:location_dataset.all_locations gs://telenor-data-science/datasets/location_dataset/*.parquet 

Copying file:///tmp/location_dataset/all_locations.parquet [Content-Type=application/octet-stream]...
/ [1 files][ 38.7 KiB/ 38.7 KiB]                                                
Operation completed over 1 objects/38.7 KiB.                                     
Waiting on bqjob_r1b00740624a4021c_0000017533a7e616_1 ... (6s) Current status: DONE   
BigQuery error in load operation: Error processing job 'telenor-data-
science:bqjob_r1b00740624a4021c_0000017533a7e616_1': Error while reading data,
error message: incompatible types for field 'year': INT32 in Parquet vs. string
in schema


In [ ]:
root_df.dtypes

KV_moving_0m_to_100m          float64
KV_moving_10000m_to_30000m    float64
KV_moving_1000m_to_3000m      float64
KV_moving_100m_to_300m        float64
KV_moving_3000m_to_10000m     float64
                               ...   
PRA_9__from5_6To7_6           float64
PRA_9__from7_6To12_5          float64
PRA_9__upTo5_6                float64
location                       object
year                           object
Length: 141, dtype: object